# Loading Data and Packages

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from gensim import corpora
from gensim import models
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LinearRegression
import numpy as np
import string
from scipy.sparse import hstack
from scipy.sparse import vstack

# fix for XGBoost errors
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

C:\Users\Tom Bliss\Anaconda3\lib\site-packages\smart_open\ssh.py:34: UserWarning: paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress
  warnings.warn('paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress')


In [2]:
import warnings
warnings.filterwarnings("ignore")

First, let's read in the dataset and take a initial look at it.

In [3]:
df_train = pd.read_csv('training_set.csv',
                       encoding = 'latin-1',
                       parse_dates = ['Created'])

df_hold = pd.read_csv('holdout_set.csv',
                      encoding = 'latin-1',
                      parse_dates = ['Created'])

In [4]:
X = df_train.drop('Engagements', axis = 1)
X['data_type'] = "training"
df_hold['data_type'] = "hold"
X = X.append(df_hold.drop('Engagements', axis = 1))

Y = df_train['Engagements']

# Feature Engineering

Next, we will create more features from our dataset mainly to capture time effect/seasonality and also use the text/captions from the posts. 

## Time Series - Month Seasonality with Trend

In [5]:
#doing this to be able to put this into linear regression
X['month'] = X.Created.apply(lambda x: x.month) #seasonal term
X['year_month'] = X.Created.apply(lambda x: x.month + x.year * 12) #trend term

## Time Series - Hourly with day of Week and Trend

In [6]:
#doing this to be able to put this into linear regression
X['hour'] = X.Created.apply(lambda x: x.hour) #seasonal term
X['weekend'] = X.Created.apply(lambda x: int(x.dayofweek >= 5)) #seasonal term
X['weekend_hour_interaction'] = X.Created.apply(lambda x: int(x.dayofweek >= 5) * x.hour) #seasonal term
X['weekday_hour_interaction'] = X.Created.apply(lambda x: int(x.dayofweek < 5) * x.hour) #seasonal term

In [7]:
#adding features
X['day_of_week'] = X.Created.apply(lambda x: x.dayofweek)
X = pd.get_dummies(X, columns = ["day_of_week"])

In [8]:
###just one hot encoding everything (not commented out)
X = pd.get_dummies(X, columns = ['hour', 'weekend', 'weekend_hour_interaction', 'weekday_hour_interaction', 'month'])

## Getting Features from text:

In [9]:
# filling NA with empty text
X.Description.fillna("", inplace = True)

In [10]:
# Initial bag-of-words method feature engineering

X['containsLink'] = X.Description.str.contains('.http').astype(float)
X['exclamationPointCount'] =X.Description.str.count('!').astype(float)
X['questionMarkCount'] = X.Description.str.count('\?').astype(float)
X['doubleQuotationMarkCount'] = X.Description.str.count('\"').astype(float)
X['singleQuoteMarkCount'] = X.Description.str.count('\'').astype(float)
X['commaMarkCount'] = X.Description.str.count(',').astype(float)
X['collinCount'] = X.Description.str.count(':').astype(float)
X['semiCollinCount'] = X.Description.str.count(';').astype(float)
X['percentMarkCount'] = X.Description.str.count('%').astype(float)
X['dollarSignCount'] = X.Description.str.count('$').astype(float)
X['hashCount'] = X.Description.str.count('#').astype(float)
X['starCount'] = X.Description.str.count('\*').astype(float)
X['atCount'] = X.Description.str.count('@').astype(float)
X['percentCapital'] = (X.Description.str.findall(r'[A-Z]').str.len().fillna(0)/X.Description.str.len().fillna(1)).fillna(0)
X['percentlowercase'] = (X.Description.str.findall(r'[a-z]').str.len().fillna(0)/X.Description.str.len().fillna(1)).fillna(0)
X['percentnumbers'] = (X.Description.str.findall(r'[0-9]').str.len().fillna(0)/X.Description.str.len().fillna(1)).fillna(0)
X['percentother'] = (1 - X['percentCapital'] - X['percentlowercase'] - X['percentnumbers']).fillna(0)

In [22]:
# use word2vec from Google News / Twitter 
w = models.KeyedVectors.load_word2vec_format(
    'GoogleNews-vectors-negative300.bin.gz', binary=True)

Using a word2vec model to extract meaning from the text.

In [23]:

texts = [[token for token in doc.translate(str.maketrans('', '', string.punctuation)).lower().split()]
               for doc in (X['Description']).astype(str)]

texts_final = []

for i in range(len(texts)):
    doc_final = []
    for j in range(len(texts[i])):
            if texts[i][j] in w:
                doc_final.append(texts[i][j])    
    if len(doc_final) < 1:
        texts_final.append(['NA'])
    else:
        texts_final.append(doc_final)
        
embedding = np.vstack([np.mean(w[doc], axis=0) for doc in texts_final])

for i in range(len(embedding[0])):
    X['embedding_' + str(i)] = embedding[:,i]

In [24]:
#creating dummies for type column
X = pd.get_dummies(X, columns = ["Type"])

Doing train test split on training data an then using tfidf vectorizor to extract each word in description indivdually.

In [25]:
X_train, X_test, y_train, y_test = train_test_split(X.loc[X.data_type == "training"].drop("data_type", axis = 1),
                                                    Y,
                                                    random_state = 23)


#count vectorizor
vect = TfidfVectorizer()
X_train_sparse = vect.fit_transform(X_train.Description)
X_test_sparse = vect.transform(X_test.Description)

X_train.drop(['Description', "Created"], axis = 1, inplace = True)
X_test.drop(['Description', "Created"], axis = 1, inplace = True)


for feature in X_train.columns:
    X_train_sparse = hstack((X_train_sparse, np.array(X_train[feature]).reshape(-1,1)))
    X_test_sparse = hstack((X_test_sparse, np.array(X_test[feature]).reshape(-1,1)))

## Fitting / Testing Model

For our model selection, we will use MAPE as our scoring system and look over both linear (LR, Lasso, etc.) and non-linear models (RandomForest, XGB). 

In [29]:
from sklearn.metrics import make_scorer
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Ridge, Lasso, LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
from xgboost import XGBRegressor

# custom MAPE scorer for sklearn
def MAPE(y, y_pred, **kwargs):
    return sum(abs((y - y_pred) / y))/len(y)

mape_scorer = make_scorer(MAPE, greater_is_better=True)

In [16]:
models = [LinearRegression(),
          Ridge(),
          Lasso(),
          RandomForestRegressor(),
          XGBRegressor()]

for mdl in models: 
    print(type(mdl).__name__)
    score = cross_val_score(mdl, X_train_sparse, y_train, 
                          n_jobs=-1, scoring=mape_scorer, cv=5)
    
    print("MAPE Scores: ", score)
    print("Mean MAPE: ", np.mean(score))
    print("\n===========================\n")

LinearRegression
MAPE Scores:  [0.14569288 0.13411596 0.12789666 0.13918109 0.14467435]
Mean MAPE:  0.13831218821204724


Ridge
MAPE Scores:  [0.32576616 0.32560614 0.32537874 0.33895551 0.32731159]
Mean MAPE:  0.32860362746736504


Lasso
MAPE Scores:  [0.06699133 0.07185476 0.068208   0.06922346 0.07171168]
Mean MAPE:  0.06959784682865824


RandomForestRegressor
MAPE Scores:  [0.05503405 0.0530034  0.05087014 0.05329416 0.05509563]
Mean MAPE:  0.05345947651825219


XGBRegressor
MAPE Scores:  [0.05108    0.05242051 0.04484114 0.04936731 0.05017491]
Mean MAPE:  0.04957677223406316




Our best performing model is XGBRegressor, but Lasso does relatively well, which we can use if we need a more interpretable/simpler model. For our case, let's just use XGBRegressor since we care more of predictive power. Let's tune the XGBoost model for our problem. 

In [18]:
best_score = 1
best_max_depth = 0
best_n_estimators = 0

for n_estimators_param in [100, 150, 250, 300]:
    for max_depth_param in [5, 7, 9, 11, 13, 15]:
            model = XGBRegressor(max_depth = max_depth_param,
                                 n_estimators = n_estimators_param)
            score = cross_val_score(model, X_train_sparse, y_train, 
                          n_jobs=-1, scoring=mape_scorer, cv=5)
        
        
            print("max_depth = ", max_depth_param)
            print("n_estimators = ", n_estimators_param)
            print("MAPE Scores: ", score)
            print("Mean MAPE: ", np.mean(score))
            print("\n===========================\n")
    
            if np.mean(score) < best_score:
                best_score = np.mean(score)
                best_max_depth = max_depth_param
                best_n_estimators = n_estimators_param



print('Best Mean CV Score: {0:.3f}'
      .format(best_score)
      + ' when max_depth = {0:.3f}'
      .format(best_max_depth)
      + ', when n_estimators = {0:.3f}'
      .format(best_n_estimators))

max_depth =  5
n_estimators =  100
MAPE Scores:  [0.03753432 0.03668317 0.03217733 0.03619439 0.0345801 ]
Mean MAPE:  0.0354338620235566


max_depth =  7
n_estimators =  100
MAPE Scores:  [0.03298479 0.03321254 0.02919338 0.03140563 0.03143909]
Mean MAPE:  0.031647084619057894


max_depth =  9
n_estimators =  100
MAPE Scores:  [0.03338576 0.03195707 0.03014845 0.03137227 0.03115694]
Mean MAPE:  0.03160409894524039


max_depth =  11
n_estimators =  100
MAPE Scores:  [0.03407896 0.03397291 0.03155152 0.03272708 0.0326472 ]
Mean MAPE:  0.03299553316992165


max_depth =  13
n_estimators =  100
MAPE Scores:  [0.0355577  0.03527928 0.03346074 0.03387516 0.03440251]
Mean MAPE:  0.034515078730822255


max_depth =  15
n_estimators =  100
MAPE Scores:  [0.0366217  0.03572249 0.03467205 0.03508342 0.0357137 ]
Mean MAPE:  0.03556267314210072


max_depth =  5
n_estimators =  150
MAPE Scores:  [0.03273732 0.03314394 0.02814703 0.03168077 0.03017431]
Mean MAPE:  0.031176673024719485


max_depth =  7


Next, lets evaluate our model on our test set.

In [30]:
xgb = XGBRegressor(max_depth = best_max_depth, n_estimators = best_n_estimators)
model = xgb.fit(X_train_sparse, y_train, verbose=True)
y_pred = xgb.predict(X_test_sparse)
print("MAPE: ", MAPE(y_test, y_pred)*100, "%")

[16:11:08] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
MAPE:  2.577589318192902 %


## Predicting on Holdout Set

With the best model above, we now predict the holdout set for submission.

In [31]:
#refitting tfidf with full data set
X_hold = X.loc[X.data_type == "hold"].drop("data_type", axis = 1)

X_train_total = X.loc[X.data_type == "training"].drop("data_type", axis = 1)

vect = TfidfVectorizer()
X_train_total_sparse = vect.fit_transform(X_train_total.Description)
X_hold_sparse = vect.transform(X_hold.Description)

X_hold.drop(['Description', "Created"], axis = 1, inplace = True)

for feature in X_train.columns:
    X_train_total_sparse = hstack((X_train_total_sparse, np.array(X_train_total[feature]).reshape(-1,1)))
    X_hold_sparse = hstack((X_hold_sparse, np.array(X_hold[feature]).reshape(-1,1)))

#refitting xgb on full data set 
model = xgb.fit(X_train_total_sparse, Y)

#double checking this was done correctly
print("Training error: ", sum(abs((Y - model.predict(X_train_total_sparse)) / Y))/len(Y))


df_hold.drop('data_type', axis = 1, inplace = True)
df_hold.Engagements = model.predict(X_hold_sparse)

[16:13:03] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
0.01262200080591515


In [32]:
df_hold.Engagements = model.predict(X_hold_sparse)

In [33]:
df_hold.to_csv("holdout_set_Columbia3.csv")